# 1. RAFT training data generation using GPT-4o

In this notebook, we'll synthesize some training data that will eventually be used to fine-tune a GPT-4o mini model in order to adapt it to a set of document(s) and specific domain. **This step is critical as the quality of your training data will greatly influence the performance of your fine-tuned model.**

For RAFT, these are the different steps to prepare the training dataset:
- Collect Domain-Specific Documents: Gather documents relevant to the domain you want to specialize the LLM in (e.g., medical documents for PubMed, legal documents, API documentation for software).
- Chunk the file into Documents
- For each Document chunk, generate a set of Questions that can be answered from the Document
- For each Document-Question pair, create a list of documents using:
    - **Golden Document (D*)**: Document that contains the answer to the question.
    - **Distractor Documents (Dk)**: Documents that do not contain relevant information.
- Question-Answer-Document Triplets: From each **Document-Question** pair, generate a factual **Answer** based on the Golden Document.

Curating a good training dataset often involves manual work and review by SMEs. That said, we can use an LLM to help us generate an initial set of training examples that can be vetted and further refined by SMEs.


### 0. Pre-requisites

For this hands-on workshop, all you need is access to an Azure subscription and the ability to create Azure OpenAI resources and deployments.

0. Install poppler for PDF processing

- on Linux run `sudo apt-get install -y poppler-utils`
- on Mac run `brew install poppler`
- on Windows run `conda install -c conda-forge poppler`

1. Create a code environment and install the necessary packages

```shell
conda create -n raft python=3.11

conda activate raft

pip install -r requirements.txt
```

2. Create a GPT-4o deployment
3. Create a GPT-4o mini deployment
4. Create an Azure OpenAI resource in North Central US or Sweden Central (regions where gpt-4o-mini fine tuning is supported)
5. Create a `.env` file based on the [sample.env](./sample.env) file in this repository to store your credentials and important environment variables. Paste your AOAI endpoints, keys and deployment names, name the file `.env`

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -r "/content/drive/MyDrive/RAFT PROJECT/azure-openai-raft-main/requirements.txt"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of multiprocess to det

In [ ]:
!pip install langchain openai pdf2image python-dotenv tqdm


**Import libraries**

In [ ]:
!pip install --upgrade openai==1.55.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.6/389.6 kB 11.0 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.40.6
    Uninstalling openai-1.40.6:
      Successfully uninstalled openai-1.40.6


In [ ]:
!pip install --upgrade openai==1.55.3
!pip install --force-reinstall -v "httpx==0.27.2"

Using pip 24.1.2 from /usr/local/lib/python3.11/dist-packages/pip (python 3.11)
  Obtaining dependency information for httpx==0.27.2 from https://files.pythonhosted.org/packages/56/95/9377bcb415797e44274b51d46e3249eba641711cf3348050f76ee7b15ffc/httpx-0.27.2-py3-none-any.whl.metadata
  Obtaining dependency information for anyio from https://files.pythonhosted.org/packages/46/eb/e7f063ad1fec6b3178a3cd82d1a3c4de82cccf283fc42746168188e1cdd5/anyio-4.8.0-py3-none-any.whl.metadata
  Obtaining dependency information for certifi from https://files.pythonhosted.org/packages/38/fc/bce832fd4fd99766c04d1ee0eead6b0ec6486fb100ae5e74c1d91292b982/certifi-2025.1.31-py3-none-any.whl.metadata
  Obtaining dependency information for httpcore==1.* from https://files.pythonhosted.org/packages/87/f5/72347bc88306acb359581ac4d52f23c0ef445b57157adedb9aee0cd689d2/httpcore-1.0.7-py3-none-any.whl.metadata
  Obtaining dependency information for idna from https://files.pythonhosted.org/packages/76/c6/c88e154df9c4e1a2a

In [ ]:
from openai import AzureOpenAI
from langchain_experimental.text_splitter import SemanticChunker
from dotenv import load_dotenv
from io import BytesIO
import base64
from typing import Literal, Any
import os
from math import ceil
import random
from tqdm import tqdm
from pdf2image import convert_from_path

load_dotenv('/content/drive/MyDrive/RAFT PROJECT/azure-openai-raft-main/sample.env')

generator_client = AzureOpenAI(
    azure_endpoint=os.getenv("AOAI_GPT4o_ENDPOINT"),
    api_version="2024-10-21",
    api_key=os.getenv("AOAI_GPT4o_API_KEY")
)

gpt4o_deployment = os.getenv("AOAI_GPT4o_DEPLOYMENT")

### 1. Loading and chunking domain-specific documents

For Retrieval Augmented Fine Tuning, we need to generate Question-Documents-Answer triplets. The first step is to create document chunks based on our domain-specific documents we want to specialize our model on.

For this workshop, we will use the publicly available [BMO Better Banking Guide](./data/better_banking_guide_en.pdf)

**Take a minute to browse through the PDF and become more familiar with its content**

The guide contains information about various banking accounts offered by BMO as well as how-to guides on e.g how to access Account statements etc.

The document is in PDF format and contains a number of tables and charts, we will use GPT-4o to convert the pages content to markdown. Credit to Liam Cavanagh for the inspiration.

**a. First we'll need to convert the document pages to images encoded in base64**

In [ ]:
!apt-get install poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 29 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.6 [186 kB]
Fetched 186 kB in 1s (358 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 124947 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.6_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.6) ...
Setting up poppler-utils (22.02.0-2ubuntu0.6) ...
Processing triggers for man-db (2.10.2-1) ...


In [ ]:
!pip install pdf2image

**b. Now we can call GPT-4o to convert our PDF to markdown**

In [ ]:
def pdf_to_base64_urls(pdf_path):
  """Converts each page of a PDF to a base64 encoded URL starting with 'data:image/jpeg'.

  Parameters:
    pdf_path: Path to the PDF file.

  Returns:
    A list of base64 encoded image URLs, one for each page.
  """

  images = convert_from_path(pdf_path)
  base64_urls = []

  for image in images:
    img_byte_arr = BytesIO()
    image.save(img_byte_arr, format="JPEG")
    img_byte_arr.seek(0)
    base64_encoded = base64.b64encode(img_byte_arr.read()).decode('utf-8')
    base64_url = f"data:image/jpeg;base64,{base64_encoded}"
    base64_urls.append(base64_url)

  return base64_urls

pdf_path = '/content/drive/MyDrive/RAFT PROJECT/azure-openai-raft-main/data/merged_output.pdf'
image_data = pdf_to_base64_urls(pdf_path)

**b. Now we can call GPT-4o to convert our images to markdown**

In [ ]:
def gpt_image_to_markdown(image_data, client):
    """
    Converts an image to markdown using GPT-4o.

    Parameters:
    image_data: A base64 encoded image.
    client: An AzureOpenAI client

    Returns:
    A list of base64 encoded image URLs, one for each page.
    """
    messages = [
        {"role":"system", "content":"""You are an AI image assistant capable of extracting text from images
         Given an image, you must extract the any visible text on the image and return it in Markdown.
         You must keep the original layout and formatting of the text as much as possible in Markdown format.
         Pay attention to the text size and use headers, subheaders, bold, italic, tables etc where necessary."""},
         {"role":"user", "content":[{
                "type":"image_url",
                "image_url":{
                    "url":image_data
                    }
         }]}
    ]
    try:
        response = client.chat.completions.create(
            model=gpt4o_deployment,
            messages=messages
        )

        return response.choices[0].message.content

    except Exception as e:
        print(f"Error: {e}")
        return None


markdown_doc = ""

with tqdm(total=len(image_data), desc="Converting images to Markdown") as pbar:
    for img_data in image_data:
        result = gpt_image_to_markdown(img_data, generator_client)
        markdown_doc += "\n" + result
        pbar.update(1)

Converting images to Markdown: 100%|██████████| 201/201 [53:29<00:00, 15.97s/it]


**c. Let's chunk the Markdown using Langchain**

We first use Langchain's `MarkdownHeaderTextSplitter` to split the document based on headers and then further split the chunks using `RecursiveCharacterTextSplitter` with a chunk size of 1024. Finally, we filter out any chunks that are too short to contain any valuable information..

In [ ]:
from langchain_text_splitters import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter
import re

def remove_special_characters(string):
    """
    Remove special characters from a string.

    Parameters:
    string (str): The input string from which special characters need to be removed.

    Returns:
    str: A new string with special characters removed.
    """
    return re.sub(r'[^a-zA-Z0-9\s]', '', string)

headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2")
]

markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on,
    strip_headers=False
    )

markdown_doc_splits = markdown_splitter.split_text(markdown_doc)
print(f"Number of chunks after markdown split: {len(markdown_doc_splits)}")

chunk_size = 1024
chunk_overlap = 50
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
)


chunked_document = text_splitter.split_documents(markdown_doc_splits)

print(f"Number of chunks after markdown + recursive split: {len(chunked_document)}")

chunks = [chunk.page_content for chunk in chunked_document if len(remove_special_characters(chunk.page_content))>100]

print(f"Number of chunks after filtering out empty: {len(chunks)}")


Number of chunks after markdown split: 108
Number of chunks after markdown + recursive split: 1083
Number of chunks after filtering out empty: 989


### 2. Generate training data from the chunked documents

We define 2 main functions to generate our Question-Document-Answer triplets from our chunked document

1. `generate_instructions_gen()`: This function generates a list of questions based on an input document chunk
2. `generate_label()`: This function generates an Answer based on a Question-Document chunk pair

**a. First, lets look at the `generate_instructions_gen()` function on a sample**

In [ ]:
def strip_str(s: str) -> str:
    """
    Helper function for helping format strings returned by GPT-4o.

    Parameters:
    s (str): The input string to be formatted.

    Returns:
    str: A formatted string
    """
    l, r = 0, len(s)-1
    beg_found = False
    for i in range(len(s)):
        if s[i].isalpha():
            if not beg_found:
                l = i
                beg_found = True
            else:
                r = i
    r += 2
    return s[l:min(r, len(s))]

def generate_instructions_gen(client: AzureOpenAI, chunk: Any, x: int = 5, model: str = None) -> list[str]:
    """
    Generates a list of questions or use cases based on a provided chunk of context using an Azure OpenAI model.

    Parameters:
    client (AzureOpenAI): An instance of the Azure OpenAI client used to communicate with the OpenAI API.
    chunk (Any): The context or chunk of text based on which the questions are to be generated.
    x (int, optional): The number of questions to generate. Default is 5.
    model (str, optional): The specific model to use for generating the questions. Default is None, which uses the default model configured in the client.

    Returns:
    list[str]: A list of generated questions.
    """
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a synthetic question-answer pair generator. Given a chunk of context about some topic(s), generate exactly %s example questions a user could ask and would be answered using information from the chunk. For example, if the given context was a Wikipedia paragraph about the United States, an example question could be 'How many states are in the United States?'" % (x)},
            {"role": "system", "content": "The questions should be able to be answered in a few words or less. Include only the questions in your response."},
            {"role": "user", "content": str(chunk)}
        ]
    )

    queries = response.choices[0].message.content.split('\n')
    queries = [strip_str(q) for q in queries]
    queries = [q for q in queries if any(c.isalpha() for c in q)]
    return queries[:int(x)]

Let's visualize an example picked randomly from our Document chunks

In [ ]:
sample_index = random.randint(0, len(chunks)-1)
chunk = chunks[sample_index]

queries = generate_instructions_gen(generator_client, chunk, x=5, model=gpt4o_deployment)

In [ ]:
print(chunk)

# **B. Carrying Subtask-Material Handling Task**
_Similar Legends_  
And # similar graph plotted like shown details repeated from  
rows outlines rows mentioned Sections
```markdown
#### Table 4
Results of the p-values from t-test in comparison of subjects with exoskeleton and without exoskeleton assessed for all subjects during stationary condition.

| **Experimental Task**                | **COPy (With Exoskeleton)** | **COPy (Without Exoskeleton)** |
|---------------------------------------|-----------------------------|---------------------------------|
| **Lifting Subtask-Material Handling** | 0.26                        | 0.35                            |
| **Lowering Subtask-Material Handling**| 0.33                        | 0.65                            |
| **Rebar Tying Task**                  | 0.83                        | 0.55                            |

*Level of significance = 0.05.*

---

#### Table 5
Results of the gait parameters assessed for all subjects during th

In [ ]:
queries


['What is the p-value for the lifting subtask with an exoskeleton?',
 'What is the p-value for the lowering subtask without an exoskeleton?',
 'What is the level of significance used in the analysis?',
 'Which task shows a COPy value of 0.55 without the exoskeleton?',
 'How does the COPy value for rebar tying compare between with and without the exoskeleton?']

**b. Generating questions, answers and adding distractor documents**

In [ ]:
from datasets import Dataset, load_dataset
import random
from typing import Any

def encode_question_gen(question: str, chunk: Any) -> list[str]:
    """
    Encode multiple prompt instructions into a single string for the general case (`pdf`, `json`, or `txt`).

    Parameters:
    question (str): The question to be answered.
    chunk (Any): The context or chunk of text that provides the information needed to answer the question.

    Returns:
    list[str]: A list of messages formatted for the language model API, including system and user roles.
    """

    prompts = []

    prompt = """
        Question: {question}\n Context: {context}\n
        Answer this question using the information given in the context above and no prior knowledge. Here is things to pay attention to:
        - First provide step-by-step reasoning on how to answer the question.
        - In the reasoning, if you need to copy paste some sentences from the context, include them in ##begin_quote## and ##end_quote##. This would mean that things outside of ##begin_quote## and ##end_quote## are not directly copy paste from the context.
        - End your response with final answer in the form <ANSWER>: $answer, the answer should be given in a joyful and friendly tone.
        - If the answer cannot be found in the context, say "I'm sorry, I cannot answer this question as I'm missing the required information"
        You MUST begin your final answer with the tag "<ANSWER>:".
    """.format(question=question, context=str(chunk))
    prompts.append({"role": "system", "content": "You are a helpful question answerer who can provide an answer given a question and relevant context."})
    prompts.append({"role": "user", "content": prompt})
    return prompts

def generate_label(client: AzureOpenAI, question: str, context: Any, model: str = None) -> str | None:
    """
    Generates the label / answer to `question` using `context` and GPT-4o.

    Parameters:
    client (AzureOpenAI): An instance of the Azure OpenAI client used to communicate with the OpenAI API.
    question (str): The question to be answered.
    context (Any): The context or chunk of text that provides the information needed to answer the question.
    model (str, optional): The specific model to use for generating the answer. Default is None, which uses the default model configured in the client.

    Returns:
    str | None: The generated answer from the language model, or None if no answer was generated.
    """
    question = encode_question_gen(question, context)
    response = client.chat.completions.create(
        model=model,
        messages=question,
        n=1,
        temperature=0
    )
    response = response.choices[0].message.content
    return response

def add_chunk_to_dataset(
    client: AzureOpenAI,
    chunks: list[str],
    chunk: str,
    x: int = 5,
    num_distract: int = 3,
    p: float = 0.8,
    model: str = None
) -> None:
    """
    Given a chunk, create {Q, A, D} triplets and add them to the dataset.

     Parameters:
    client (AzureOpenAI): An instance of the Azure OpenAI client used to communicate with the OpenAI API.
    chunks (list[str]): A list of chunks of text from which distractor documents can be sampled.
    chunk (str): The chunk of text to use as the primary context for generating questions and answers.
    x (int, optional): The number of questions to generate for the given chunk. Default is 5.
    num_distract (int, optional): The number of distractor documents to include with each question. Default is 3.
    p (float, optional): The probability of including the oracle (original) document as part of the context. Default is 0.8.
    model (str, optional): The specific model to use for generating questions and answers. Default is None, which uses the default model configured in the client.
    """
    global ds
    global errors
    i = chunks.index(chunk)
    try:
        qs = generate_instructions_gen(client, chunk, x, model)
    except Exception as e:
        errors.append(e)
        return None
    for q in qs:
        datapt = {
            "id": None,
            "type": None,
            "question": None,
            "context": None,
            "oracle_context": None,
            "cot_answer": None
        }

        datapt["id"] = f"seed_task_{i}"
        datapt["type"] = "general"
        datapt["question"] = q

        # add num_distract distractor docs
        docs = [chunk]
        indices = list(range(0, len(chunks)))
        indices.remove(i)
        for j in random.sample(indices, num_distract):
            docs.append(chunks[j])

        # decides whether to add oracle document
        oracle = random.uniform(0, 1) < p
        if not oracle:
            docs[0] = chunks[random.sample(indices, 1)[0]]
        random.shuffle(docs)

        d = {
            "title": [],
            "sentences": []
        }

        d["title"].append(["placeholder_title"]*(num_distract+1))
        d["sentences"].append(docs)
        datapt["context"] = d
        datapt["oracle_context"] = chunk

        # add answer to q
        try:
            datapt["cot_answer"] = generate_label(client, q, chunk, model=model)
        except Exception as e:
            errors.append(e)
            continue

        # construct model instruction
        context = ""
        for doc in docs:
            context += "<DOCUMENT>" + str(doc) + "</DOCUMENT>\n"
        context += q
        datapt["instruction"] = context

        # add to dataset
        if not ds:
            # init ds
            datapt["id"] = [datapt["id"]]
            datapt["type"] = [datapt["type"]]
            datapt["question"] = [datapt["question"]]
            datapt["context"] = [datapt["context"]]
            datapt["oracle_context"] = [datapt["oracle_context"]]
            datapt["cot_answer"] = [datapt["cot_answer"]]
            datapt["instruction"] = [datapt["instruction"]]
            ds = Dataset.from_dict(datapt)
        else:
            ds = ds.add_item(datapt)

**Let's execute this function a in multi-threaded way to speed up the process**

In [ ]:
from tqdm import tqdm
import concurrent.futures

errors = []
ds = Dataset.from_dict({})


def process_chunk(chunk):
    add_chunk_to_dataset(generator_client, chunks, chunk, 5, 3, model=gpt4o_deployment)

# Create a ThreadPoolExecutor with the desired number of workers
with concurrent.futures.ThreadPoolExecutor() as executor:
    # Submit the tasks to the executor and store the Future objects
    futures = [executor.submit(process_chunk, chunk) for chunk in chunks]

    # Use tqdm to create a progress bar
    with tqdm(total=len(chunks), desc="Processing chunks") as pbar:
        # Iterate over the completed futures as they become available
        for future in concurrent.futures.as_completed(futures):
            # Get the result of the completed future
            result = future.result()
            # Update the progress bar
            pbar.update(1)

# Print any errors that occurred during processing
print(f'Number of processing errors: {errors}/{len(chunks)}')

Processing chunks: 100%|██████████| 989/989 [2:33:09<00:00,  9.29s/it]

Number of processing errors: [RateLimitError("Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-10-21 have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 34 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.'}}"), RateLimitError("Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-10-21 have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 1 second. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.'}}"), RateLimitError("Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-10-21 have exceeded token rate limit of your curren

In [ ]:
training_df = ds.to_pandas()

print(f'{training_df.shape[0]} rows and {training_df.shape[1]} columns in the training dataset')

3275 rows and 7 columns in the training dataset


In [ ]:
# Previewing the generated data

training_df.head(2)

,id,type,question,context,oracle_context,cot_answer,instruction
0,seed_task_3,general,How many facilitators were identified in the s...,{'sentences': [['# EPiC Series in Built Enviro...,**Results:** Important factors included 18 fac...,"To answer the question ""How many facilitators ...",<DOCUMENT># EPiC Series in Built Environment\n...
1,seed_task_2,general,What framework was used to guide the study on ...,"{'sentences': [['- Di Marco, R., Rubega, M., L...",**Method:** A literature review was conducted ...,Step-by-step reasoning:\n\n1. The question ask...,"<DOCUMENT>- Di Marco, R., Rubega, M., Lennon, ..."


**c. Formatting the data in chat format for fine tuning with Azure OpenAI**

The conversational chat format is required to fine-tune gpt-4o-mini

In [ ]:
training_df["messages"] = training_df.apply(lambda x: [
                                                     {"role":"user", "content":x['instruction']},
                                                     {"role":"assistant", "content":x['cot_answer']}
                                                     ], axis=1)

In [ ]:
training_df.messages.values[12]

In [ ]:
training_df.dropna(subset=['cot_answer'], inplace=True)

### 2. Spitting our data into training and test sets

Splitting your data into training, validation and testing sets when fine-tuning a large language model (LLM) is crucial for ensuring the model's performance and generalization capabilities. The training set is used to teach the model, allowing it to learn patterns from the data. The validation set is used to track performance metrics during the training to avoid underfitting / overfitting. However, to objectively evaluate how well the model has learned and to ensure it can generalize to unseen data, a separate testing set is necessary.

We will use this test set in order to measure the improvement of performance we get from using RAFT over RAG with gpt-4o-mini

In [ ]:
import numpy as np

train_df, validate_df, test_df = np.split(
    training_df.sample(frac=1, random_state=42),
                       [int(.8*len(training_df)), int(.9*len(training_df))]
                       )

print(f"Train: {train_df.shape[0]}, Validate: {validate_df.shape[0]}, Test: {test_df.shape[0]}")

Train: 2620, Validate: 327, Test: 328


/usr/local/lib/python3.11/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [ ]:
import os

# Set your desired path here
save_path = "/content/drive/MyDrive/RAFT PROJECT/azure-openai-raft-main/data/training_data"

# Ensure the directory exists
if not os.path.exists(save_path):
    os.makedirs(save_path)

# Save the files in the specified directory
train_df[['messages']].to_json(os.path.join(save_path, "exo_train.jsonl"), orient="records", lines=True)
test_df.to_json(os.path.join(save_path, "exo_test.jsonl"), orient="records", lines=True)
validate_df[['messages']].to_json(os.path.join(save_path, "exo_validation.jsonl"), orient="records", lines=True)

print(f"Files saved to {save_path}")


Files saved to /content/drive/MyDrive/RAFT PROJECT/azure-openai-raft-main/data/training_data


#### Congrats! We now have a labelled training dataset and a test dataset to evaluate our model's performance. Now go to the [finetuning notebook](./raft_finetuning.ipynb)